<a href="https://colab.research.google.com/github/vgaquino/capstone-team-29/blob/main/model_evaluation_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook is used to compare the similarity between labels from Amazon Rekognition and the Vizwiz captions using  

Method 1. TF-IDF representation and cosine similarity.  
Method 1. BoW representation and Jaccard similarity.  

**Need to update the correct AWS Rekognition result to evaluate.**

In [ ]:
import pandas as pd
import os
import json
import zipfile

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt') # for tokenization
nltk.download('stopwords') # for removing stopword
nltk.download('wordnet')     # for lemmatization
stop_words = set(stopwords.words('english'))

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Need to be updated with correct dataset.** becasue for now, I use the NodeXL image-labels data as a temporary representation of the image-labels data AWS Rekognition processed for experiment purpose.  

In [ ]:
file_path = '/content/drive/My Drive/Capstone Project/NodeXL_tweets_labels.csv'
aws_df=pd.read_csv(file_path)

In [ ]:
aws_df.head()

,Unnamed: 0,Confidence Threshold,Labels,Image Name,Image URL,Media in Tweet,Media Type,Tweet URL
0,0,False,['Blackboard'],FQPvyP0XsBYT0_e.jpg,https://pbs.twimg.com/media/FQPvyP0XsBYT0_e.jpg,https://t.co/WpzDllqKhf https://pbs.twimg.com/media/FQPvyP0XsBYT0_e.jpg,photo,https://t.co/WpzDllqKhf
1,1,False,"['Outdoors', 'Cross', 'Nature']",FpQm4VQXwAEh29s.jpg,https://pbs.twimg.com/media/FpQm4VQXwAEh29s.jpg,https://t.co/lu3TqALtrs https://pbs.twimg.com/media/FpQm4VQXwAEh29s.jpg,photo,https://t.co/lu3TqALtrs
2,2,True,"['Adult', 'Male', 'Man', 'Person', 'Clothing', 'Formal Wear', 'Suit', 'Text', 'Face', 'Head']",FpsDtlRXwAIkgL4.jpg,https://pbs.twimg.com/media/FpsDtlRXwAIkgL4.jpg,https://t.co/xSfLH7zAdn https://pbs.twimg.com/media/FpsDtlRXwAIkgL4.jpg,photo,https://t.co/xSfLH7zAdn
3,3,False,"['Advertisement', 'Astronomy', 'Outer Space']",GD30009W8AASCYQ.jpg,https://pbs.twimg.com/media/GD30009W8AASCYQ.jpg,https://t.co/7mBSR5nCmW https://pbs.twimg.com/media/GD30009W8AASCYQ.jpg,photo,https://t.co/7mBSR5nCmW
4,4,True,"['City', 'Urban', 'Metropolis', 'Architecture', 'Building', 'Cityscape', 'Person', 'Car', 'Transportation', 'Vehicle', 'Road', 'Street']",GDVdH1qX0AMv1Wd.jpg,https://pbs.twimg.com/media/GDVdH1qX0AMv1Wd.jpg,https://t.co/KgWWt5rA4V https://pbs.twimg.com/media/GDVdH1qX0AMv1Wd.jpg,photo,https://t.co/KgWWt5rA4V


In [ ]:
# creation of aws image-labels df
aws_df = aws_df[['Unnamed: 0', 'Labels']].rename(columns={"Unnamed: 0": "image_id"})

In [ ]:
aws_df.head()

,image_id,Labels
0,0,['Blackboard']
1,1,"['Outdoors', 'Cross', 'Nature']"
2,2,"['Adult', 'Male', 'Man', 'Person', 'Clothing', 'Formal Wear', 'Suit', 'Text', 'Face', 'Head']"
3,3,"['Advertisement', 'Astronomy', 'Outer Space']"
4,4,"['City', 'Urban', 'Metropolis', 'Architecture', 'Building', 'Cityscape', 'Person', 'Car', 'Transportation', 'Vehicle', 'Road', 'Street']"


Preprocess aws labels

In [ ]:
def preprocess_text(text):
    # lowercase
    text = text.lower()

    # tokenize text
    tokens = word_tokenize(text)

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    no_stopwords_tokens = [word for word in tokens if word not in stop_words]

    # Lemmatize text
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in no_stopwords_tokens]

    text = ' '.join(lemmatized_tokens)

    return text

In [ ]:
# Apply preprocessing
aws_df['preprocessed_text'] = aws_df['Labels'].apply(preprocess_text)

aws_df.head()

,image_id,Labels,preprocessed_text
0,0,['Blackboard'],[ 'blackboard ' ]
1,1,"['Outdoors', 'Cross', 'Nature']","[ 'outdoors ' , 'cross ' , 'nature ' ]"
2,2,"['Adult', 'Male', 'Man', 'Person', 'Clothing', 'Formal Wear', 'Suit', 'Text', 'Face', 'Head']","[ 'adult ' , 'male ' , 'man ' , 'person ' , 'clothing ' , 'formal wear ' , 'suit ' , 'text ' , 'face ' , 'head ' ]"
3,3,"['Advertisement', 'Astronomy', 'Outer Space']","[ 'advertisement ' , 'astronomy ' , 'outer space ' ]"
4,4,"['City', 'Urban', 'Metropolis', 'Architecture', 'Building', 'Cityscape', 'Person', 'Car', 'Transportation', 'Vehicle', 'Road', 'Street']","[ 'city ' , 'urban ' , 'metropolis ' , 'architecture ' , 'building ' , 'cityscape ' , 'person ' , 'car ' , 'transportation ' , 'vehicle ' , 'road ' , 'street ' ]"


Load validation data from VizWiz

In [ ]:
#extract annotations (val.json, train.json, test.json)
zip_files = ['/content/drive/MyDrive/Capstone Project/annotations.zip']

#destination folder for extracted files
destination_folder = '/content/'

#create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

#loop through each zip file
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        #extract the contents of the zip file to the destination folder
        zip_ref.extractall(destination_folder)

In [ ]:
with open('annotations/val.json', 'r') as f:
    annotation_data = json.load(f)

annotation_data.keys()

dict_keys(['info', 'images', 'annotations'])

In [ ]:
len(annotation_data['images'])

7750

In [ ]:
annotation_data['annotations'][:2]

[{'caption': 'A computer screen shows a repair prompt on the screen.',
  'image_id': 23431,
  'is_precanned': False,
  'is_rejected': False,
  'id': 117155,
  'text_detected': True},
 {'caption': 'a computer screen with a repair automatically pop up',
  'image_id': 23431,
  'is_precanned': False,
  'is_rejected': False,
  'id': 117156,
  'text_detected': True}]

In [ ]:
caption_list = []
id_list = []
for captions in annotation_data['annotations']:
    caption_list.append(captions['caption'])
    id_list.append(captions['image_id'])

In [ ]:
viz_df = pd.DataFrame(data={'caption': caption_list, 'image_id': id_list})

In [ ]:
viz_df.head()

,caption,image_id
0,A computer screen shows a repair prompt on the screen.,23431
1,a computer screen with a repair automatically pop up,23431
2,partial computer screen showing the need of repairs,23431
3,Part of a computer monitor showing a computer repair message.,23431
4,The top of a laptop with a blue background and dark blue text.,23431


In [ ]:
viz_df['image_id'] = viz_df['image_id'] - 23431 # adjust image id and image name

In [ ]:
viz_df.head(10)

,caption,image_id
0,A computer screen shows a repair prompt on the screen.,0
1,a computer screen with a repair automatically pop up,0
2,partial computer screen showing the need of repairs,0
3,Part of a computer monitor showing a computer repair message.,0
4,The top of a laptop with a blue background and dark blue text.,0
5,A person is holding a bottle that has medicine for the night time.,1
6,A bottle of medication has a white twist top.,1
7,night time medication bottle being held by someone,1
8,a person holding a small black bottle of NIGHT TIME,1
9,A bottle of what appears to be cough syrup held in hand.,1


In [ ]:
# For each image, combine the five captions into a single list
viz_combined = viz_df.groupby('image_id')['caption'].apply(' '.join).reset_index()

viz_combined.rename(columns={'caption': 'combined_captions'}, inplace=True)

In [ ]:
# Apply preprocessing
viz_combined['preprocessed_text'] = viz_combined['combined_captions'].apply(preprocess_text)

In [ ]:
viz_combined.head()

,image_id,combined_captions,preprocessed_text
0,0,A computer screen shows a repair prompt on the screen. a computer screen with a repair automatically pop up partial computer screen showing the need of repairs Part of a computer monitor showing a computer repair message. The top of a laptop with a blue background and dark blue text.,computer screen show repair prompt screen . computer screen repair automatically pop partial computer screen showing need repair part computer monitor showing computer repair message . top laptop blue background dark blue text .
1,1,A person is holding a bottle that has medicine for the night time. A bottle of medication has a white twist top. night time medication bottle being held by someone a person holding a small black bottle of NIGHT TIME A bottle of what appears to be cough syrup held in hand.,person holding bottle medicine night time . bottle medication white twist top . night time medication bottle held someone person holding small black bottle night time bottle appears cough syrup held hand .
2,2,"a white paper showing an image of black and brown dog A library book with pictures of two dogs on the cover on a wooden table. A book with a black and a tan dog walking down a snowy street. The book cover shows two dogs in the snow A book cover title Dog Years with an image of a black and brown dog walking up the street, on the left side it has a due date sticker from a library.","white paper showing image black brown dog library book picture two dog cover wooden table . book black tan dog walking snowy street . book cover show two dog snow book cover title dog year image black brown dog walking street , left side due date sticker library ."
3,3,"A white box is to the left of a blue box on a wooden table. A small rectangular red and white box next to a small rectangular blue box on a wooden surface. two boxes of medicine, one white and red and the other blue sitting on a table Two boxes that appear to contain medication or eye drops Two boxes of pharmaceutical products left in a table","white box left blue box wooden table . small rectangular red white box next small rectangular blue box wooden surface . two box medicine , one white red blue sitting table two box appear contain medication eye drop two box pharmaceutical product left table"
4,4,close up of a computer monitor that is powered on. A monitor has a message displayed on it. Pictured here is a screenshot that shows an error message from an app. Computer screen displaying an error saying the display driver is not supported by Zoom Text. a screenshot of someone's monitor that is having issues,close computer monitor powered . monitor message displayed . pictured screenshot show error message app . computer screen displaying error saying display driver supported zoom text . screenshot someone 's monitor issue


## Method 1: TF-IDF representation & Cosine similarity

Create a TF-IDF representation for preprocessed tex column in both aws_df and viz_combined dataframes.

In [ ]:
# define a tf-idf function

def create_tfidf(dataframe, column):
    tfidf_vectorizer = TfidfVectorizer()

    tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe[column])

    feature_names = tfidf_vectorizer.get_feature_names_out()

    return tfidf_matrix, feature_names

In [ ]:
tfidf_aws, features_aws = create_tfidf(aws_df, 'preprocessed_text')
tfidf_viz_combined, features_viz_combined = create_tfidf(viz_combined, 'preprocessed_text')

In [ ]:
print("TF-IDF Representation (DataFrame aws_df):", tfidf_aws.toarray())
print("TF-IDF Feature Names (DataFrame aws_df):", features_aws)

print("TF-IDF Representation (DataFrame viz_combined):", tfidf_viz_combined.toarray())
print("TF-IDFFeature Names (DataFrame viz_combined):", features_viz_combined)

TF-IDF Representation (DataFrame aws_df): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
TF-IDF Feature Names (DataFrame aws_df): ['accessories' 'acrobatic' 'activity' 'adapter' 'adult' 'adventure'
 'advertisement' 'aerial' 'aid' 'aircraft' 'airfield' 'airliner'
 'airplane' 'airport' 'alcohol' 'alloy' 'amaryllidaceae' 'animal'
 'antelope' 'antenna' 'apartment' 'applause' 'apple' 'appliance' 'aquatic'
 'arch' 'architecture' 'arena' 'arm' 'arrangement' 'art' 'astronomy'
 'audience' 'auditorium' 'auto' 'baby' 'backpack' 'badge' 'bag' 'balcony'
 'ball' 'balloon' 'banner' 'bar' 'barbie' 'baseball' 'basketball'
 'bathroom' 'beach' 'beak' 'beard' 'bed' 'bell' 'belt' 'beverage'
 'bicycle' 'bird' 'black' 'blackboard' 'blade' 'blazer' 'blonde' 'blouse'
 'blue' 'board' 'boardwalk' 'boat' 'body' 'book' 'bottle' 'bouquet' 'box'
 'boy' 'bread' 'brick' 'bridge' 'brown' 'building' 'bull' 'bulld

Compute Cosine Similarity Scores for corresponding images

In [ ]:
# Compute cosine similarity (the result is between each pair of images)
cosine_sim_matrix = cosine_similarity(tfidf_aws, tfidf_viz_combined)

# To get similarity scores for corresponding images, need to extract diagonal elements
corresponding_similarity_scores = cosine_sim_matrix.diagonal()

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 642 while Y.shape[1] == 9433

In [ ]:
for index, score in enumerate(corresponding_similarity_scores):
    print(f"Cosine similarity between AWS Rekognition and VizWiz Groundtruth of image {index} is: {score:.3f}")

NameError: name 'corresponding_similarity_scores' is not defined

## Method 2: BoW representation & Jaccard similarity

In [ ]:
# Bag of Words representation

def create_BoW(dataframe, column):
    vectorizer = CountVectorizer(binary=True)
    BoW_matrix = vectorizer.fit_transform(dataframe[column])

    feature_names = vectorizer.get_feature_names_out()

    return BoW_matrix, feature_names

In [ ]:
BoW_aws, features_aws = create_BoW(aws_df, 'preprocessed_text')
BoW_viz_combined, features_viz_combined = create_BoW(viz_combined, 'preprocessed_text')

In [ ]:
print("BoW Representation (DataFrame aws_df):", BoW_aws.toarray())
print("Feature Names (DataFrame aws_df):", features_aws)

print("BoW Representation (DataFrame viz_combined):", BoW_viz_combined.toarray())
print("Feature Names (DataFrame viz_combined):", features_viz_combined)

BoW Representation (DataFrame aws_df): [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Feature Names (DataFrame aws_df): ['accessories' 'acrobatic' 'activity' 'adapter' 'adult' 'adventure'
 'advertisement' 'aerial' 'aid' 'aircraft' 'airfield' 'airliner'
 'airplane' 'airport' 'alcohol' 'alloy' 'amaryllidaceae' 'animal'
 'antelope' 'antenna' 'apartment' 'applause' 'apple' 'appliance' 'aquatic'
 'arch' 'architecture' 'arena' 'arm' 'arrangement' 'art' 'astronomy'
 'audience' 'auditorium' 'auto' 'baby' 'backpack' 'badge' 'bag' 'balcony'
 'ball' 'balloon' 'banner' 'bar' 'barbie' 'baseball' 'basketball'
 'bathroom' 'beach' 'beak' 'beard' 'bed' 'bell' 'belt' 'beverage'
 'bicycle' 'bird' 'black' 'blackboard' 'blade' 'blazer' 'blonde' 'blouse'
 'blue' 'board' 'boardwalk' 'boat' 'body' 'book' 'bottle' 'bouquet' 'box'
 'boy' 'bread' 'brick' 'bridge' 'brown' 'building' 'bull' 'bulldozer'
 'business' 'butcher' 'cabinet' 'cable' 

In [ ]:
jaccard_similarities = []
for i in range(BoW_aws.shape[0]):
  ja_similarity = jaccard_score(BoW_aws.toarray()[i], BoW_viz_combined.toarray()[i], average='binary')
  jaccard_similarities.append(ja_similarity)
  print(f"Jaccard similarity between AWS Rekognition and VizWiz Groundtruth of image {i} is {ja_similarity:.3f}")

ValueError: Found input variables with inconsistent numbers of samples: [642, 9433]